# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 1 ноября 2018, 06:00 
**Штраф за опоздание:** -2 балла после 06:00 1 ноября, -4 балла после 06:00 8 ноября, -6 баллов после 06:00 15 ноября

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (4 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы fit такой, чтобы она была медленнее sklearn не более чем в 10 раз. Скорость проверяем на  wine и Speed Dating Data. Для ускорения используем только numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 3 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [74]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [75]:
# %%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
            self.G_function_for_y = self.__y_gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
            self.G_function_for_y = self.__y_entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
            self.G_function_for_y = self.__y_misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - np.sum((l_c ** 2 / l_s + r_c ** 2 / r_s) / (l_s + r_s),
                          axis=1)

    def __y_gini(self, y):
        return 1 - np.sum((np.bincount(y) / y.size) ** 2)

    def __entropy(self, l_c, l_s, r_c, r_s):
        return -np.sum((l_c * np.log2(l_c/l_s) + r_c *
                        np.log2(r_c/r_s)) / (l_s + r_s), axis=1)

    def __y_entropy(self, y):
        return -np.sum(((np.bincount(y)/y.size) *
                        np.log2(np.bincount(y)/y.size)))

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - np.max(l_c / (l_s + r_s), axis=1) -\
                    np.max(r_c / (l_s + r_s), axis=1)

    def __y_misclass(self, y):
        return 1 - np.max(np.bincount(y) / y.size)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:np.sqrt(n_feature)]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:np.log2(n_feature)]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # Сортирует по убыванию значения x, y переставляет в том же порядке
        # Запоминает количество классов
        # Определяет сколько нужно отбросить(срезать) справа и слева от y
        # чтобы осталось не менее min_samples_split элементов
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        cut_size = np.int(self.min_samples_split / 2 - 1)

        # Что делает этот блок кода?
        # Производит срез y, если cut_size ненулевой
        # Определяет номера элементов, где в срезе происходит смена класса,
        # так, чтобы в каждом разбиении оказалось не менее min_samples_split
        # элементов
        # Если таких элементов нет, возвращает бесконечность,
        # как степень неопределенности и None вместо лучше элемента для деления
        if cut_size == 0:
            splitted_sorted_y = sorted_y
        else:
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] + (cut_size + 1)

        if len(r_border_ids) == 0:
            return np.inf, None

        # Что делает этот блок кода?
        # Определяет количество одинаковых элементов в разбиении
        # Кодирует разбиение one-hot-кодом, 1 -- если класс поменялся,
        # 0 -- если нет
        # Сохраняет матрицу количества классов для разбиения
        eq_el_count = r_border_ids - np.append(np.array([cut_size]),
                                               r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:cut_size], minlength=class_number)

        # Что делает этот блок кода?
        # Считает количество классов в левой и правой части разбиения
        # Считает количество элементов в правой и левой части разбиения
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y,
                                    minlength=class_number) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # Считает неопределенность, выбирает индекс с минимальной
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # Что делает этот блок кода?
        # Возвращает степень неопределенности и элемент, по которому
        # нужно делить для такой степени неопределенности
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node
        if (self.max_depth is not None and self.max_depth < depth) or\
                (y.size < self.min_samples_split) or\
                (np.unique(y).shape[0] == 1) or\
                (np.bincount(y).argmax() > y.size * self.sufficient_share):
            self.__create_leaf(node_id, y)
            return

        feature_ids = self.get_feature_ids(x.shape[1])
        thresholds = np.array([self.__find_threshold(x[:, feature_id], y)
                               for feature_id in feature_ids])

        best_feature = feature_ids[np.argmin(thresholds[:, 0])]
        _, threshold = thresholds[np.argmin(thresholds[:, 0])]

        x_l, x_r, y_l, y_r = self.__div_samples(x, y, best_feature, threshold)

        if y_l.size == 0 or y_r.size == 0:
            self.__create_leaf(node_id, y)
            return

        self.__create_non_leaf(node_id, best_feature, threshold)
        self.feature_importances_[best_feature] +=\
            self.G_function_for_y(y) * y.size -\
            self.G_function_for_y(y_l) * y_l.size -\
            self.G_function_for_y(y_r) * y_r.size
        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1, best_feature)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1, best_feature)
        return

    def __create_leaf(self, node_id, y):
        self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                              np.bincount(y) / y.size)
        return

    def __create_non_leaf(self, node_id, feature_id, threshold):
        self.tree[node_id] = (self.NON_LEAF_TYPE, feature_id, threshold)
        return

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= np.sum(self.feature_importances_)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [76]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [77]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [78]:
%time clf.fit(X_train, y_train)

CPU times: user 2.54 ms, sys: 1.29 ms, total: 3.83 ms
Wall time: 2.13 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [79]:
%time my_clf.fit(X_train, y_train)

CPU times: user 25.5 ms, sys: 7.42 ms, total: 32.9 ms
Wall time: 27 ms


## Проверка качества работы на wine

In [80]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9440559440559441

In [81]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

1.0

## Подготовка данных Speed Dating Data 

In [82]:
# тут делаете то же самое, что и на семинаре 
# https://github.com/stroykova/spheremailru/blob/master/2018-02/lecture_04_trees/pract-speed-dating-trees-proc.ipynb
df = pd.read_csv('./Speed Dating Data.csv', encoding='latin1')
df = df.iloc[:, :97]
df = df.drop(['id', 'idg', 'condtn', 'round', 'position', 'positin1',
             'order', 'partner', 'age_o', 'race_o', 'pf_o_att', 
              'pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o', 'undergra',
              'from', 'zipcode', 'sports','tvsports','exercise','dining',
              'museums','art','hiking','gaming', 'clubbing','reading','tv',
              'theater','movies','concerts','music','shopping','yoga',
              'expnum'], axis=1)
df.drop_duplicates(subset=['iid']).gender.value_counts()
df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(19)
df = df.drop(['field'], axis=1)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(df.mn_sat[df.mn_sat.notna()].mean())

df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(df.tuition[df.tuition.notna()].mean())

df = df.dropna(subset=['imprelig', 'imprace'])
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)

df = df.dropna(subset=['date'])

df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(18)
df = df.drop(['career'], axis=1)

df.loc[:, 'temp_totalsum'] = df.loc[:,
                                    ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1',
                                     'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
    (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T
     / df.loc[:, 'temp_totalsum'].T).T * 100
df.loc[:, 'temp_totalsum'] = df.loc[:, 
                                    ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
                                     'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T
     / df.loc[:, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)

for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i), 
            'intel{}_1'.format(i), 'fun{}_1'.format(i), 
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    
    df = df.drop(feat, axis=1)
df = df.drop(['wave'], axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1)\
                                 .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                                   .dropna()
        
df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'),
                       on='pid',
                       how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

## Проверка скорости работы на Speed Dating Data 

In [83]:
clf = DecisionTreeClassifier(min_samples_split=2)
my_clf = MyDecisionTreeClassifier(min_samples_split=2)

In [84]:
%time clf.fit(X_train, y_train)

CPU times: user 87.1 ms, sys: 2.28 ms, total: 89.4 ms
Wall time: 87.7 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [85]:
%time my_clf.fit(X_train, y_train)

CPU times: user 825 ms, sys: 7.6 ms, total: 833 ms
Wall time: 832 ms


## Проверка качества работы на Speed Dating Data

In [86]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5475681789619202

In [87]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5738333688472192

## Задание 3

In [88]:
features = df_pair.columns[1:]

In [89]:
pd.Series(data=clf.feature_importances_,
          index=features).sort_values(ascending=False).head(10)

int_corr     0.058318
age_f        0.033922
shar1_1_f    0.033526
income_f     0.029852
amb1_1_f     0.029470
tuition_f    0.027890
amb3_1_f     0.026507
sinc1_1_f    0.025221
fun1_1_f     0.024769
fun1_1       0.024026
dtype: float64

In [90]:
pd.Series(data=my_clf.feature_importances_,
          index=features).sort_values(ascending=False).head(10)

int_corr     0.086566
amb1_1_f     0.081034
age          0.065287
race_f       0.050699
attr3_1      0.049016
mn_sat_f     0.044807
fun2_1_f     0.041362
date         0.040963
imprelig     0.036300
tuition_f    0.034437
dtype: float64

## Задание 4

In [91]:
rfc = RandomForestClassifier(n_jobs=-1, n_estimators=10)

param_grid = { 
    'max_depth': [1,30],
    'min_samples_leaf': range(1,30, 2),
    'criterion': ['gini', 'entropy']
}

CV = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
CV.fit(X, y)
print(CV.best_params_)
print(CV.best_score_)

{'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 1}
0.8364591147786947
